# Actividad PBL 1

### César Isao Pastelin Kohagura - A01659947

### Sophia Gabriela Martínez Albarrán - A01424430

### Luis Emilio Fernández González - A01659517

### Eduardo Botello Casey - A01659281

In [ ]:
import numpy as np
import pandas as pd

import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk

from sklearn.model_selection import train_test_split

In [31]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sophiagabrielamartinezalbarran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/sophiagabrielamartinezalbarran/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [32]:
data = pd.read_csv('spam.csv', encoding='latin-1')
data.drop([data.columns[col] for col in [2, 3, 4]], axis=1, inplace=True)

In [33]:
#Limpiamos los datos, los tokenizamos y los regresamos como una lista de palabras
def procesarEmail(contents):
    ps = PorterStemmer()

    contenido = contents.lower()
    contenido = re.sub(r'<[^<>]+>', ' ', contenido)
    contenido = re.sub(r'[0-9]+', 'number', contenido)
    contenido = re.sub(r'(http|https)://[^\s]*', 'httpaddr', contenido)
    contenido = re.sub(r'[^\s]+@[^\s]+', 'emailaddr', contenido)
    contenido = re.sub(r'[$]+', 'dollar', contenido)

    palabras = word_tokenize(contenido)

    for i in range(len(palabras)):
        palabras[i] = re.sub(r'[^a-zA-Z0-9]', '', palabras[i])
        palabras[i] = ps.stem(palabras[i])

    palabras = [word for word in palabras if len(word) >= 1]

    return palabras

In [34]:
#Recibimos la lista de las palabras usadas en todos los emails 
# y creamos un vocabulario con las palabras más frecuentes
def getVocabulario(emails, longitud_vocabulario):
    vocabulario = dict()
    
    for i in range(len(emails)):
        emails[i] = procesarEmail(emails[i])
        for palabra in emails[i]:
            if palabra in vocabulario.keys():
                vocabulario[palabra] += 1
            else:
                vocabulario[palabra] = 1

    vocabulario = sorted(vocabulario.items(), key=lambda x: x[1], reverse=True)
    vocabulario = list(map(lambda x: x[0], vocabulario[0:longitud_vocabulario]))
    vocabulario = {index: word for index, word in enumerate(vocabulario)}

    return vocabulario

In [35]:
# Recorrimos el diccionario buscando la clave de un valor específico
def getllave(dictionario, val):
    for key, value in dictionario.items():
        if value == val:
            return key

In [36]:
# Devolvemos los índices de las palabras del vocabulario que aparecen en un email
def getIndices(email, vocabulario):
    indice_palabras = set()
    
    for palabra in email:
        if palabra in vocabulario.values():
            indice_palabras.add(getllave(vocabulario, palabra))

    return indice_palabras

In [37]:
# Vectorización del email para el modelo
def obtenerVectorCaracteristico(indice_palabras, longitud_vocabulario):
    VectorCaracteristico = np.zeros(longitud_vocabulario)

    for i in indice_palabras:
        VectorCaracteristico[i] = 1

    return VectorCaracteristico

In [38]:
longitud_vocabulario= 2000

In [40]:
# Construímos el vocabulario con las palabras más frecuentes
vocabulario = getVocabulario(data['v2'].to_list(), longitud_vocabulario)

# Limpiamos y tokenizamos todos los correos
emails = data['v2'].to_list()
emails = list(map(lambda x: procesarEmail(x), emails))

### Implementacion del Naive Bayes

In [46]:
class NaiveBayes():
    def __init__(self):
        self.probabilidad_clase = dict()
        self.probabilidad_palabra = dict()
        self.clases = []
    
    def fit(self, X, y):
        # Obtenemos las clases únicas, en este caso spam y ham
        self.clases = np.unique(y)
        
        # Guardamos la probabilidad de cada clase y la probabilidad de cada palabra dado una clase en un diccionario para luego poder hacer las predicciones
        for c in self.clases:
            X_c = X[y == c]
            self.probabilidad_clase[c] = X_c.shape[0] / X.shape[0]
            
            # Suavizado de Laplace: Se usa para evitar probabilidades de cero, ya que si predecimos un nuevo email que no contiene una palabra del vocabulario 
            # la probabilidad de esa palabra sería cero, lo cual afectaría el cálculo de la probabilidad total por lo que asumimos que todas las palabras
            # al menos aparecen una vez en cada clase
            self.probabilidad_palabra[c] = (X_c.sum(axis=0) + 1) / (X_c.shape[0] + 2)
    
    def predecir(self, X):
        predicciones = []
        
        # Predecimos la clase para cada email
        for i in range(X.shape[0]):
            prediciones_por_clase = dict()
            
            # Probabilidad a priori de cada clase
            for c in self.clases:
                # Vamos a usar logaritmos para evitar underflow y para que las probabilidades sean más manejables, tambien para evitar
                # multiplicar y mejor sumar, log(a x b) = log(a) + log(b)
                probabilidad_clase_c = np.log(self.probabilidad_clase[c])
                
                # Tomamos en cuenta la probabilidad de cada palabra en el email dado la clase, pero también la probabilidad de que no aparezca
                # ya que ambas cosas nos dan información sobre la clase del email
                probabilidad = X.iloc[i] * np.log(self.probabilidad_palabra[c]) + (1 - X.iloc[i]) * np.log(1 - self.probabilidad_palabra[c])
                probabilidad = probabilidad.sum()
                probabilidad_final = probabilidad_clase_c + probabilidad
                prediciones_por_clase[c] = probabilidad_final
            
            # Tomamos la clase con la probabilidad más alta, es decir si P(clase|email) es mayor para spam que para ham, entonces el email es spam
            predicciones.append(max(prediciones_por_clase, key=prediciones_por_clase.get))
        
        return np.array(predicciones)

    def predecir_proba(self, X):
        probabilidades = []
        
        for i in range(X.shape[0]):
            # Calculamos P(email|clase) × P(clase) para cada clase
            numeradores = dict()
            
            for c in self.clases:
                # P(clase) - probabilidad a priori
                #Es la proporción de correos spam o ham en el dataset
                priori = self.probabilidad_clase[c]
                
                # P(email|clase) - probabilidad
                # Si la palabra aparece en el email entonces se va a multiplicar P(palabra|clase)
                # Si la palabra no aparece en el email se multiplica por (1 - P(palabra|clase))
                probabilidad = 1.0
                for j in range(len(X.iloc[i])):
                    if X.iloc[i].iloc[j] == 1:  # palabra presente
                        probabilidad *= self.probabilidad_palabra[c].iloc[j]
                    else:  # palabra ausente
                        probabilidad *= (1 - self.probabilidad_palabra[c].iloc[j])
                
                # P(email|clase) × P(clase)
                #Es la probabilidad conjunta de que el email y la clase ocurran juntos.
                numeradores[c] = probabilidad * priori

            # P(email) = suma de todos los numeradores
            #Es la probabilidad total del email sin importar la clase.
            #Se necesita para normalizar y que las probabilidades finales sumen 1.
            # Sale de la ley de probabilidad total
            # P(email) = P(email|ham)×P(ham) + P(email|spam)×P(spam)
            denominador = sum(numeradores.values())
            
            # P(clase|email) = P(email|clase) × P(clase) / P(email)
            # Esto nos da la probabilidad de que el email pertenezca a cada clase (Spam o Ham)
            probabilidades_reales = [numeradores[c] / denominador for c in sorted(self.clases)]
            probabilidades.append(probabilidades_reales)
            
        return np.array(probabilidades)

In [42]:
X = list(map(lambda x: obtenerVectorCaracteristico(getIndices(x, vocabulario), longitud_vocabulario), emails))
X = pd.DataFrame(np.array(X).astype(np.int16))
y = data['v1'] 

## Entrenamiento y prueba del modelo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo
modelo = NaiveBayes()
modelo.fit(X_train, y_train)


In [45]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Predicciones
y_pred = modelo.predecir(X_test)
y_proba = modelo.predecir_proba(X_test)

# Métricas básicas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='spam')
recall = recall_score(y_test, y_pred, pos_label='spam')
f1 = f1_score(y_test, y_pred, pos_label='spam')

print("=== MÉTRICAS DE EVALUACIÓN ===")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall : {recall:.4f}")
print(f"F1-Score : {f1:.4f}")

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred, labels=['ham', 'spam'])

print("\n=== MATRIZ DE CONFUSIÓN ===")
print("                Predicho")
print("              Ham    Spam")
print(f"Real   Ham    {cm[0,0]:4d}    {cm[0,1]:4d}")
print(f"       Spam   {cm[1,0]:4d}    {cm[1,1]:4d}")


=== MÉTRICAS DE EVALUACIÓN ===
Accuracy : 0.9812
Precision : 0.9640
Recall : 0.8933
F1-Score : 0.9273

=== MATRIZ DE CONFUSIÓN ===
                Predicho
              Ham    Spam
Real   Ham     960       5
       Spam     16     134
